# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [7]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [8]:
from langchain.indexes import VectorstoreIndexCreator

In [9]:
#pip install docarray

In [10]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [11]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [12]:
response = index.query(query)

In [13]:
display(Markdown(response))

 I'm sorry, I don't know.

In [14]:
loader = CSVLoader(file_path=file)

In [15]:
docs = loader.load()

In [16]:
docs[0]

Document(page_content='Product Name: Hiking Jacket\nCategory: Hiking Gear\nPrice: 89.99\nColor: Green\nSize: Large\nDescription: Waterproof and windproof hiking jacket with breathable fabric\nRating: 4.5\nReviews: Great jacket! Keeps me dry in rainy hikes.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [17]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [18]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [19]:
print(len(embed))

1536


In [20]:
print(embed[:5])

[-0.021913960576057434, 0.006774206645786762, -0.018190348520874977, -0.039148248732089996, -0.014089343138039112]


In [21]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [22]:
query = "Please suggest a shirt with sunblocking"

In [23]:
docs = db.similarity_search(query)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-KMVwMLshTdfKEkN6IMIH3AJO on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


In [24]:
len(docs)

4

In [25]:
docs[0]

Document(page_content='Product Name: Cycling Jersey\nCategory: Cycling Gear\nPrice: 49.99\nColor: Blue\nSize: Small\nDescription: Breathable and moisture-wicking cycling jersey for long rides\nRating: 4.4\nReviews: Love the design and fabric quality.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 7})

In [26]:
retriever = db.as_retriever()

In [27]:
llm = ChatOpenAI(temperature = 0.0)


In [28]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [29]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [30]:
display(Markdown(response))

| Product Name   | Category     | Price | Color  | Size  | Description                                                  | Rating | Reviews                                                      |
|----------------|--------------|-------|--------|-------|--------------------------------------------------------------|--------|--------------------------------------------------------------|
| Cycling Jersey | Cycling Gear | 49.99 | Blue   | Small | Breathable and moisture-wicking cycling jersey for long rides | 4.4    | Love the design and fabric quality.                          |
| Rain Poncho    | Outdoor Gear | 24.99 | Yellow | One Size | Compact and lightweight rain poncho for outdoor activities | 4.1    | Keeps me dry during unexpected rain showers.                 |
| Insulated Jacket | Outdoor Gear | 119.99 | Gray | Medium | Insulated jacket for cold weather conditions with a removable hood | 4.9    | Perfect for winter hiking and skiing.                        |
| Hiking Jacket  | Hiking Gear  | 89.99 | Green  | Large | Waterproof and windproof hiking jacket with breathable fabric | 4.5    | Great jacket! Keeps me dry in rainy hikes.                   |

Summary:
1. Cycling Jersey: A breathable and moisture-wicking jersey designed for long rides. It has a rating of 4.4 and customers love its design and fabric quality.
2. Rain Poncho: A compact and lightweight rain poncho suitable for outdoor activities. It has a rating of 4.1 and customers appreciate its ability to keep them dry during unexpected rain showers.
3. Insulated Jacket: An insulated jacket for cold weather conditions, featuring a removable hood. It has a high rating of 4.9 and is highly recommended for winter hiking and skiing.
4. Hiking Jacket: A waterproof and windproof jacket designed for hiking, with breathable fabric. It has a rating of 4.5 and customers find it great for keeping them dry during rainy hikes.

In [31]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [32]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [33]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [34]:
display(Markdown(response))

| Product Name    | Category     | Price | Color | Size  | Description                                                  | Rating | Reviews                                      |
|-----------------|--------------|-------|-------|-------|--------------------------------------------------------------|--------|----------------------------------------------|
| Cycling Jersey  | Cycling Gear | 49.99 | Blue  | Small | Breathable and moisture-wicking cycling jersey for long rides | 4.4    | Love the design and fabric quality.          |
| Hiking Jacket   | Hiking Gear  | 89.99 | Green | Large | Waterproof and windproof hiking jacket with breathable fabric | 4.5    | Great jacket! Keeps me dry in rainy hikes.   |
| Insulated Jacket| Outdoor Gear | 119.99| Gray  | Medium| Insulated jacket for cold weather conditions with a removable hood | 4.9 | Perfect for winter hiking and skiing.        |

Summary:
1. Cycling Jersey: A breathable and moisture-wicking cycling jersey for long rides. It has a rating of 4.4 and customers love the design and fabric quality.
2. Hiking Jacket: A waterproof and windproof hiking jacket with breathable fabric. It has a rating of 4.5 and customers find it great for keeping dry during rainy hikes.
3. Insulated Jacket: An insulated jacket for cold weather conditions with a removable hood. It has a high rating of 4.9 and is perfect for winter hiking and skiing.

In [35]:
response = index.query(query, llm=llm)

In [36]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])